In [1]:
import numpy as np

import json
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.models import *
from keras.layers import *
from keras.callbacks import *
from keras.optimizers import *
import keras.backend as K
import tensorflow as tf

/home/fabio/miniconda3/envs/dl/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
with open('data/sentipolc_word_index.json', 'r') as f:
    word_index = json.load(f)

In [3]:
with open('data/words.tsv', 'w') as f:
    for w,_ in word_index.items():
        f.write('{}\n'.format(w))

In [4]:
words = [w for w,_ in word_index.items()]

In [5]:
data = np.load('data/sentipolc_seq.npz')

x_train = data['x_train']
y_train = data['y_train']

x_test = data['x_test']
y_test = data['y_test']

Check the max length of the text

In [6]:
max_len_seq = max([len(x) for x in x_train])
print('max len seq {}'.format(max_len_seq))
max_idx = max(np.array(x_train).max())
print('max id {}'.format(max_idx))

max len seq 40
max id 10208


In [7]:
x_train_pad = sequence.pad_sequences(x_train, maxlen=max_len_seq)
x_test_pad = sequence.pad_sequences(x_test, maxlen=max_len_seq)

In [8]:
x_train_pad.shape

(4154, 40)

In [9]:
x_train_pad[:2]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  457,
          10, 1105,   15,   58, 1106,   22, 3781,    1,  104,    7,  642,
          35,   31,   16,    1,   12,   58,    2],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,  807, 1107,
         931,  458,   16,    4,   12,   58,    2]], dtype=int32)

# Define metrics

In [10]:
def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.
     Computes the precision, a metric for multi-label classification of
     how many selected items are relevant.
      """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))		
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))		
    precision = true_positives / (predicted_positives + K.epsilon())		
    return precision

In [11]:
def recall(y_true, y_pred):
    """Recall metric.
 
    Only computes a batch-wise average of recall.
    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

In [12]:
from sklearn.model_selection import StratifiedShuffleSplit

split_train_val = StratifiedShuffleSplit(1,test_size=0.2, random_state=123456)

for train, test in split_train_val.split(x_train_pad, y_train):
    x_train_pad_split, y_train_pad_split = x_train_pad[train], y_train[train]
    x_val_pad_split, y_val_pad_split = x_train_pad[test], y_train[test]

In [13]:
callbacks = [
    EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='auto'),
    ReduceLROnPlateau(patience=2, verbose=1)
]

In [16]:
def build_model():
    model = Sequential()
    model.add(Embedding(input_dim=max_idx+1, output_dim=100, input_length=max_len_seq))
    model.add(LSTM(64,  dropout=0.2, recurrent_dropout=0.2, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
    
    return model

In [17]:
model = build_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 40, 100)           1020900   
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                42240     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                650       
_________________________________________________________________
dropout_2 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 11        
Total params: 1,063,801
Trainable params: 1,063,801
Non-trainable params: 0
_________________________________________________________________


In [18]:
history = model.fit(x_train_pad, y_train, validation_split=0.2,
                    batch_size=128, epochs=5)

Train on 3323 samples, validate on 831 samples
Epoch 1/5
3323/3323 [==============================] - 3s 975us/step - loss: 0.6810 - acc: 0.6277 - val_loss: 0.6946 - val_acc: 0.4934
Epoch 2/5
3323/3323 [==============================] - 1s 437us/step - loss: 0.6309 - acc: 0.6422 - val_loss: 0.6756 - val_acc: 0.4934
Epoch 3/5
3323/3323 [==============================] - 1s 438us/step - loss: 0.5709 - acc: 0.6657 - val_loss: 0.6373 - val_acc: 0.6534
Epoch 4/5
3323/3323 [==============================] - 1s 435us/step - loss: 0.4716 - acc: 0.7788 - val_loss: 0.5845 - val_acc: 0.7329
Epoch 5/5
3323/3323 [==============================] - 1s 438us/step - loss: 0.3569 - acc: 0.8682 - val_loss: 0.6086 - val_acc: 0.7377


In [ ]:
del model

## Evaluate on the test set

In [19]:
model = build_model()

history = model.fit(x_train_pad, y_train, batch_size=128, epochs=4)

Epoch 1/4
4154/4154 [==============================] - 3s 630us/step - loss: 0.6831 - acc: 0.6290
Epoch 2/4
4154/4154 [==============================] - 2s 410us/step - loss: 0.6283 - acc: 0.6979
Epoch 3/4
4154/4154 [==============================] - 2s 415us/step - loss: 0.5300 - acc: 0.7747
Epoch 4/4
4154/4154 [==============================] - 2s 411us/step - loss: 0.3913 - acc: 0.8698


In [20]:
loss, acc = model.evaluate(x_test_pad, y_test)

1050/1050 [==============================] - 1s 683us/step


In [21]:
print('loss {}, acc {}'.format(loss, acc))

loss 0.65830677878289, acc 0.6819047622453599


In [22]:
#model.save('models/sentipolc_word_lstm.hdf5')

# Overall metrics

In [25]:
from sklearn import metrics

predictions = model.predict_classes(x_test_pad, batch_size=128)
tn, fp, fn, tp = metrics.confusion_matrix(y_test, predictions).ravel()

recall = tp/(tp+fp)
precision = tp/(tp+fn)
f1 = metrics.f1_score(y_test,predictions)

print ('precision: {}'.format(precision), '\nrecall: {}'.format(recall), '\nf1: {}'.format(f1))

precision: 0.6740506329113924 
recall: 0.4797297297297297 
f1: 0.5605263157894737
